In [ ]:
# In this exercise, we want to explore supervised learning the way we did in class, and build models
# we can run on a micro-controller like the ESP 32. To run this exercise, you will need Python 3.x and
# Jupyter notebook. One way is to use Jupyter online (see at https://jupyter.org), another one is to 
# install Jupyter notebook on your computer, a third way is to install the Anaconda navigator on your computer.
# This third way is of course better suited for those who will want to work further with ML
# (see at https://anaconda.org/anaconda/anaconda-navigator). Choose a way and proceed with Jupyter.
# Note: in Jupyter, 'enter' gets you another line in the same block. "Shift enter" executes the current block.

In [ ]:
#First, let's import some libraries we will need for our computation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [ ]:
#Next, let's import our training set. In your case, the path will be different, modify to match your path to unconv_MV_v5.csv
df = pd.read_csv('/Users/jerhenry/Documents/Perso/IMT/IMT_ML_IoT/unconv_MV_v5.csv');

In [ ]:
# A good first step is to look at your data, we plot an 18 by 12 figure
plt.figure(figsize=(18,12))
# we plot with the x axis taken from the viscosity, Por, and the y axis taken from the pressure, Prod
plt.plot(df[['Por']], df[['Prod']], 'o')
# And we add caption and legend
plt.title("Optimal Pump Pressure Measurements")
plt.xlabel("Viscosity (Por)")
plt.ylabel("Pressure (Prod)")

In [ ]:
#Another good step is to look at your data, the 'df' we created above, head gives us the first 5 lines, but you can 
# put another number, for example df.head(10) would show the first 10 entries
df.head()

In [ ]:
# Moving into gradient descent. We first create a shorthand notation, x for the viscosity, y for the pressure column 
x = np.array(df[['Por']])
y = np.array(df[['Prod']])
# we also extract the lentgh of these columns, so we can run the loop on all 'n' entries in the columns
n = len(df[['Por']])
# We pick up two initial values for theta0 and theta1. You could use random values, but it is common to start with 0s
th0_curr = th1_curr = 0
# We also need to decide how many times wil will run the loop. It is common to start with something like 1000, then refine later
iterations = 1000
#then, we need to decide by how much we change theta0 and theta1, for now let's use a fixed number, something small
learning_rate = 0.002

In [ ]:
# then we run our loop, for the number of iterations we decided above
for i in range(iterations):
    #at each step,  we take the x value, and use our theta0 and theta1 to predict some y value (likely wrong at the beginning)
    y_predicted = th1_curr * x + th0_curr
    #as we need to modify a bit theta0 and theta1 at each step, we calculate (at each step), the derivative of each theta
    dth1 = -(2/n)*sum(x*(y - y_predicted))
    dth0 = -(2/n)*sum(y - y_predicted)
    #then our next theta is going to be changed by the value of the derivative times the learning rate. Think about what happens here:
    # if the derivative is positive (we are going down toward 0, which is our goal, as we reach a minimum when the derivative is 0)
    # then the next value of theta will be a bit smaller than the previous one. If the derivative is negative (we are too low),
    # then the next value of theta will be a bit larger than the previous one (going back up toward 0).
    # At the same time, as the derivative gets closer to 0, we change theta by a smaller and smaller value, to avoid missing the minimum
    th1_curr = th1_curr - learning_rate * dth1
    th0_curr = th0_curr - learning_rate * dth0
    # one good way to see what is going on is to print at each iteration the thetas and the cost
    # if everything works well, then the cost should be going down. So we don't need the cost for the loop itself,
    # but we want to compute it here, just so we can print it and see if it is going down:
    cost = (1/n) * sum ([val**2 for val in (y - y_predicted)])
    print("th1 {}, th0 {}, cost {}, iteration {}".format(th1_curr,th0_curr,cost,i))
    # if the cost is going down too slow, use a larger learning rate. If the cost is not going down, your learning rate is too large
    # Here, try with a learning rate of 0.1 (bounces around the minumum, too large), and 0.0001 (too slow)

In [ ]:
# Let's plot our data again, and overlay there the line we found
# As our x values range from 5 to 25, we just compute two points on the line, one at x=5.5 and the other at x=24.5 (we compute the predicted
# y for each, now that we have our thetas). The below code is ugly, but the goal is to show you what happens, even if you do not master python:
A1 = 5.5
B1 = int(th1_curr * A1 + th0_curr)
A2 = 24.5 
B2 = int(th1_curr * A2 + th0_curr)
P1 = [A1, A2]
P2 = [B1, B2]
# the same figure as before:
plt.figure(figsize=(18,12))
plt.plot(df[['Por']], df[['Prod']], 'o')
plt.title("Optimal Pump Pressure Measurements")
plt.xlabel("Viscosity (Por)")
plt.ylabel("Pressure (Prod)")
# adding our line:
plt.plot(P1, P2)

In [ ]:
# Now let's automate all this instead, by using scikit learn libraries, in particular the linear model:
from sklearn import linear_model
# let's create a linear regression object
reg = linear_model.LinearRegression()

In [ ]:
# then all the training we did above is summarized in a single command, taking as parameters the x and y columns we train against:
reg.fit(df[['Por']],df[['Prod']])

In [ ]:
# All done. Let's plot the whole thing again:
plt.figure(figsize=(18,12))
plt.plot(df[['Por']], df[['Prod']], 'o')
plt.title("Optimal Pump Pressure Measurements")
plt.xlabel("Viscosity (Por)")
plt.ylabel("Pressure (Prod)")
# adding our predicted line, this time in green (again, there is  a better way, using this heavy handed for clarity):
A3 = 5.5
B3 = int(reg.predict([[A3]]))
A4 = 24.5 
B4 = int(reg.predict([[A4]]))
P3 = [A3, A4]
P4 = [B3, B4]
plt.plot(P3, P4, color = 'green')


In [ ]:
# If you want to see the thetas, theta0 is called the intercept, and theta1 is called the coefficient:
int(reg.intercept_), int(reg.coef_)
# 1. What coefficient did you find?

In [ ]:
# Last, if you want to run a prediction, you can use the same 'predict' command, for example suppose a new Por value:
New_Por = 8
#let's predict the presure for that Por:
reg.predict([[New_Por]])
# 2. What predicted value do you find?

In [ ]:
# A good part of ML is trying to understand the data. Por is related to Prod, but it is also related to another parameter.
# Spend some time graphing Por as x against the other columns (as y), you will find one of them that also
# displays a linear relationship with Por. Besides Por (molecular porosity) and Prod (production output)
# that you already know, the set includes Perm (permeability, how well water can mix with the oil), AI
# (accoustic impedance, how well sound traverses the product), Brittle (brittleness of hard particles),
# TOC (total organic carbon), and VR (reflectance).
# 3. Which other linear relationship did you find to Por? Graph it and find the coefficient and the intercept. 

In [ ]:
#Now let's try multivariate, also factoring TOC.
plt.figure(figsize=(18,12))
# A fist step is to look at if POC is also linear with Prod
plt.plot(df[['TOC']], df[['Prod']], 'o')

In [ ]:
# It is linear, we can thus set our model with 2 variables
reg.fit(df[['Por', 'TOC']],df[['Prod']])

In [ ]:
#Let's look at our coefficients and intercept:
int(reg.intercept_), reg.coef_
# 4. Which coefficient, intercept did you find?

In [ ]:
# syntax for 3-D projection
fig1 = plt.figure(figsize=(18,12))
ax = plt.axes(projection ='3d')
ax.scatter(df[['Por']], df[['TOC']], df[['Prod']])
ax.set_title('Optimal pressure based on Por and TOC')
ax.set_xlabel("Viscosity (Por)")
ax.set_ylabel("Granularity (TOC)")
ax.set_zlabel("Pressure (Prod)")
A5 = 5.5
B5 = -0.2
C5 = int(reg.predict([[A5, B5]]))
A6 = 24.5 
B6 = 2.2
C6 = int(reg.predict([[A6, B6]]))
P5 = [A5, A6]
P6 = [B5, B6]
P7 = [C5, C6]
ax.plot(P5, P6, P7)
plt.show()

In [ ]:
# 5. Reproduce the same process with the other value that you found correlated to Por. Graph the relationship
# What coefficients and intercept did you find?

In [ ]:
#Next, you may want to export your model
import joblib
# Save your model to a file - you should see that file in your working directory
joblib.dump(reg, 'my_cool_joblib_model')

In [ ]:
# How big is the model file? If it is 20Kb or less, it should fit onto an ESP 32.
# On your target IoT device running python, you can put that model file, then then load the model. I give it a different name than 'reg'
# so you can see how to call it:
mj = joblib.load('my_cool_joblib_model')

In [ ]:
#Then you can use that model to predict values just like before:
mj.predict(([[5.5, -0.2]]))

In [ ]:
# We can also attempt to run logistic regression on this data. First, let's look at our data:
plt.figure(figsize=(18,12))
plt.plot(df[['Brittle']], df[['Reuse']], 'o')
plt.title("Pumps issues based on grains brittleness")
plt.xlabel("Brittleness")
plt.ylabel("1 if pump could be reused, 0 if it was clogged")

In [ ]:
# For linear or logistic regression, it is good practice to split the data between training and test set. 
# Let's first do that, keeping 20% for testing:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df[['Brittle']], df.Reuse, train_size=0.8)

In [ ]:
# Let's do logistic regression. We use directly the sklearn function:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [ ]:
# Then we apply the function on our data, with the fit call:
model.fit(X_train, Y_train)

In [ ]:
# We can look at our theta coefficients:
 model.intercept_, model.coef_
# What coefficient and intercept did you find?

In [ ]:
# We can then look at our test set, and run prediction on it:
X_test

In [ ]:
# then you can predict which entries in the test set will be prediected for 1 and for 0.
# 6. How many are prediected for 1, and how many for 0?
 model.predict(X_test)

In [ ]:
# We can also look at the probability values, instead of merely looking at the prediction (yes/no) result.
# The ouput displays the probability for 0 in the first column, and the probability for 1 in the second :
model.predict_proba(X_test)
# Is the output coherent with the prediction made above?

In [ ]:
# to illustrate how the prediction works by projecting the probability onto a curve, we can generate
# brittleness values (from 10 to 85, by jumps of 0.5), then plot the prediction:
brittleness = np.arange(10, 85, 0.5)
probabilities= []
for i in brittleness:
    p_clogs = model.predict_proba([[i]])
    probabilities.append(p_clogs[:,1])
plt.scatter(brittleness,probabilities)
plt.title("Logistic Regression Model")
plt.xlabel('Brittleness')
plt.ylabel('Status (0: clogged, 1: reused)')

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
X = df.values
X.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
from sklearn. decomposition import PCA
pca_30 = PCA (n_components=30, random_state=2020)
pca_30.fit (X_scaled)
X_pca_30 = pca_30.transform(X_scaled)

In [ ]:
print ("Variance explained by all 30 principal components =",
sum (pca_30.explained_variance_ratio_ * 100))

In [ ]:
pca_30.explained_variance_ratio_ * 100

In [ ]:
plt.plot(np.cumsum(pca_30.explained_variance_ratio_ ))
plt.xlabel ('Number of components')
plt.ylabel ('Explained variance')
plt. savefig('elbow_plot.png', dpi=100)

In [ ]:
print("Variance explained by the First principal component =",
np. cumsum(pca_30.explained_variance_ratio_ * 100)[0])
print ("Variance explained by the First 2 principal components =",
np. cumsum (pca_30.explained_variance_ratio_ * 100)[1])
print ("Variance explained by the First 3 principal components =",
np.cumsum(pca_30.explained_variance_ratio_* 100) [2])
print ("Variance explained by the First 10 principal components =",
np. cumsum (pca_30.explained_variance_ratio_ * 100) [9])

In [ ]:
pca_2 = PCA(n_components=2, random_state=2020)
pca_2.fit(X_scaled)
X_pca_2 = pca_2.transform(X_scaled)

In [ ]:
pca_95 = PCA (n_components=0.95, random_state=2020)
pca_95.fit(X_scaled)
X_pca_95 = pca_95.transform(X_scaled)

In [ ]:
df_new = pd.DataFrame (X_pca_95, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])
df_new['label'] = cancer.target
df_new.head()

In [ ]:
df.head()